In [1]:
import asyncio
import boto3
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json


In [2]:
#LLM 생성
class AdvancedLLM:
    """고급 LLM 스트리밍 관리자"""
    
    def __init__(self, region_name: str = "us-west-2"):
        self.region_name = region_name
        self.model_id = None
        self.bedrock_client = self._setup_bedrock_client()
        self.llm = self._setup_llm()
    
    def _setup_bedrock_client(self):
        """Bedrock 클라이언트 설정"""
        try:
            return boto3.client(
                service_name='bedrock-runtime',
                region_name=self.region_name
            )
        except Exception as e:
            print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
            return None
    
    def _setup_llm(self, model_id="global.anthropic.claude-sonnet-4-20250514-v1:0"):
        """LLM 설정"""
        try:
            if not self.bedrock_client:
                print("❌ Bedrock 클라이언트가 없습니다.")
                return None
            self.model_id = model_id
            return ChatBedrock(
                client=self.bedrock_client,
                model_id=self.model_id,
                model_kwargs={
                    "max_tokens": 2000,
                    "temperature": 0.15,
                    "top_p": 0.9,
                }
            )
        except Exception as e:
            print(f"❌ LLM 초기화 실패: {e}")
            return None

In [3]:

agent = None


In [4]:
#Bedroc AgenticCore 유저의 요청 (payload)를 받고 Claude LLM에게 요청해서 결과를 유저에게 다시 전달해주는 로직
async def extract_text(payload):
    """텍스트 추출 AgentCore Runtime 엔트리포인트"""
    global agent


    #langchain에서 LLM 생성
    if agent is None:
        yield {"type": "status", "message": "🚀 LLM 초기화 중..."}
        agent = AdvancedLLM()
    
    # payload에서 입력 데이터 추출
    user_input = payload.get("input_data", "태양의 온도에 대해 말해줘")
    
    # 체인 구성
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant."),
        ("human", user_input)
    ])
    
    chain = prompt | agent.llm | StrOutputParser()
    
    yield {"type": "status", "message": "🔥 응답 생성 중..."}
    
    collected_text = []
    try:
        async for event in chain.astream_events({}):
            if event["event"] == "on_chat_model_stream":
                chunk = event["data"]["chunk"]
                if hasattr(chunk, 'content') and chunk.content:
                    if isinstance(chunk.content, str):
                        collected_text.append(chunk.content)
                        yield {"type": "stream", "content": chunk.content}
                    elif isinstance(chunk.content, list):
                        for content_item in chunk.content:
                            if isinstance(content_item, dict):
                                if text := content_item.get('text'):
                                    collected_text.append(text)
                                    yield {"type": "stream", "content": text}
                            elif isinstance(content_item, str):
                                collected_text.append(content_item)
                                yield {"type": "stream", "content": content_item}
                                
    except Exception as e:
        yield {"type": "error", "message": f"❌ 스트리밍 실패: {e}"}
        

    
    # 최종 결과
    final_text = "".join(collected_text)
    yield {"type": "final", "content": final_text}

In [5]:
#client

In [6]:
async for result in extract_text({"input_data": "달의 온도는?"}):
    if result['type'] == 'status':
        print(result['message'])
    elif result['type'] == 'stream':
        print(result['content'], end='')
    elif result['type'] == 'final':
        print(f"\n\n최종 결과:\n{result['content']}")
    elif result['type'] == 'error':
        print(f"오류: {result['message']}")

🚀 LLM 초기화 중...
🔥 응답 생성 중...
달의 온도는 위치와 시간에 따라 극단적으로 변합니다:

## 주요 온도 범위
- **낮 시간 (태양 직사광)**: 약 127°C (260°F)
- **밤 시간**: 약 -173°C (-280°F)

## 특별한 지역들
- **극지방 분화구**: -230°C (-382°F) 이하
- **적도 지역**: 온도 변화가 가장 극심

## 온도 변화의 원인
1. **대기 부재**: 열을 보존할 대기가 없음
2. **긴 낮과 밤**: 각각 약 14일간 지속
3. **낮은 열전도율**: 달 표면 물질이 열을 잘 전달하지 못함

이러한 극한 온도 차이로 인해 달은 태양계에서 가장 온도 변화가 심한 천체 중 하나입니다.

최종 결과:
달의 온도는 위치와 시간에 따라 극단적으로 변합니다:

## 주요 온도 범위
- **낮 시간 (태양 직사광)**: 약 127°C (260°F)
- **밤 시간**: 약 -173°C (-280°F)

## 특별한 지역들
- **극지방 분화구**: -230°C (-382°F) 이하
- **적도 지역**: 온도 변화가 가장 극심

## 온도 변화의 원인
1. **대기 부재**: 열을 보존할 대기가 없음
2. **긴 낮과 밤**: 각각 약 14일간 지속
3. **낮은 열전도율**: 달 표면 물질이 열을 잘 전달하지 못함

이러한 극한 온도 차이로 인해 달은 태양계에서 가장 온도 변화가 심한 천체 중 하나입니다.
